In [10]:
import pandas as pd

price_df = pd.read_csv("漲跌.csv")
news_df = pd.read_csv("新聞.csv")

df_news.head(3)
df_up


,名稱,年月日,漲跌
0,台積電,2023/1/3,1
1,台積電,2023/1/4,-1
2,台積電,2023/1/5,1
3,台積電,2023/1/6,0
4,台積電,2023/1/9,1
...,...,...,...
510,台積電,2025/2/21,1
511,台積電,2025/2/24,-1
512,台積電,2025/2/25,-1
513,台積電,2025/2/26,1


In [13]:
# 重新命名兩個 DataFrame
df_news = news_df.copy()
df_up = price_df.copy()

# 建立新的欄位：漲跌標籤（1：漲，0：平，-1：跌）
def label_trend(change):
    if change > 0:
        return 1
    elif change < 0:
        return -1
    else:
        return 0

# 確保 df_up 日期為 datetime 格式
df_up["年月日"] = pd.to_datetime(df_up["年月日"], format="%Y/%m/%d")
df_up["漲跌標籤"] = df_up["漲跌"].apply(label_trend)

# 將 df_up 的日期欄位往前移一天，表示「前一日新聞 → 隔日股價」
# 看漲看跌是根據隔日(N+1)實際狀況
df_up["對應新聞日"] = df_up["年月日"] - pd.Timedelta(days=1)

# 製作查找表：{新聞日: 隔日漲跌標籤}
trend_lookup = dict(zip(df_up["對應新聞日"].dt.strftime("%Y/%m/%d"), df_up["漲跌標籤"]))

# 對 df_news 做 mapping
df_news["post_time"] = pd.to_datetime(df_news["post_time"])
df_news["預計漲跌"] = df_news["post_time"].dt.strftime("%Y/%m/%d").map(trend_lookup)

df_news.head(10)


,post_time,title,author,content,預計漲跌
0,2023-01-03 10:23:00,台積電不給力 台股早盤陷萬四關卡保衛戰,李娟萍／台北報導,"2023年首個交易日，台股加權指數3日開在14,108.16點，權王台積電（2330）開在4...",-1.0
1,2023-01-03 13:50:00,台積電重返450元！台股強攻翻紅 指數漲86點收14224點,三立新聞網,記者戴玉翔／台北報導台股今（3）日迎來 2023 年交易首日，開盤出師不利，盤中一度跌至 1...,-1.0
2,2023-01-04 13:51:00,台積電壓尾盤 台股收跌24.99點14200失守,The Central News Agency 中央通訊社,（中央社記者韓婷婷台北4日電）台股今天在晶圓雙雄穩盤，百貨、造紙、光電、儲能、網通等族群衝鋒...,1.0
3,2023-01-05 11:08:00,台積電帶領半導體股勁揚 台股盤中漲逾百點,The Central News Agency 中央通訊社,（中央社記者張建中台北5日電）美股收高，費城半導體指數勁揚逾2%。台股今天在台積電股價反彈，...,0.0
4,2023-01-05 16:35:00,去年此時台積電上看1000元！謝金河揪戰犯：這件事毀滅行情,邱怡萱,台股近期在14300點關卡陷入震盪，財信傳媒董事長謝金河在臉書發文回顧台股行情，去年的今天，...,0.0
5,2023-01-06 09:01:00,升息擔憂再起 台股小跌近30點 台積電挫低3.5元,郭宜欣,升息疑慮再起，周四美股收黑，台股今（6日）以14278.52點開出，下跌逾20點，權值股台積...,NaN
6,2023-01-06 18:17:00,麥克瘋／台積電法說未演先轟動 法人保守看待,NaN,台積電12日召開法說會，近期市場消息面相當多，幾乎一面倒看淡今年前兩季營收、獲利表現。部分外...,NaN
7,2023-01-08 04:10:00,台積電小金雞 股價挑戰6字頭 采鈺、金居 錢景看俏,鄭郁平／台北報導,台股在新年首周開紅盤，摩爾投顧分析師張貽程指出，外資近期心態已漸由空翻多，在其買盤加持下，大...,1.0
8,2023-01-08 12:21:00,台積電法說會本週登場 法人：台股首季重要風向球,The Central News Agency 中央通訊社,（中央社記者賴言曦台北8日電）台股6日開低震盪，終場上漲72.29點，週線終止連4黑。展望本...,1.0
9,2023-01-09 08:24:00,美股勁揚有利台股 本週仍待觀察台積電法說會,The Central News Agency 中央通訊社,（中央社記者潘智義台北9日電）美國去年12月工資上調幅度以及新增就業成長溫和，降低市場對美聯...,1.0


In [15]:
# 將 NaN 的“預計漲跌”列去掉
df_news = df_news.dropna(subset=["預計漲跌"])

# 確保「預計漲跌」是 int 型別
df_news["預計漲跌"] = df_news["預計漲跌"].astype(int)


Index(['post_time', 'title', 'author', 'content', '預計漲跌'], dtype='object')

In [24]:
df_news.shape

# 共7973筆資料

(7973, 5)

In [25]:
df_news["預計漲跌"].value_counts()

# 4194比看漲文章，3199筆看跌文章，580筆持平文章（不做分析）

預計漲跌
 1    4194
-1    3199
 0     580
Name: count, dtype: int64

In [28]:
df_news.to_csv("預期漲跌.csv", index=False, encoding="utf-8-sig")